# Set a correct format the tables generated as CoxPH reports

## 1. Libraries

In [2]:
import pandas as pd
import openpyxl 
import os
import re
import docx

## 2. Functions

In [3]:
def reformat_excel(folder_path, excel_name):
    excel_file_path = folder_path + excel_name
    
    # Open the file
    workbook = openpyxl.load_workbook(excel_file_path)
    sheet = workbook.active

    # Get the headers
    headers_cell = [cell for cell in sheet[1]]

    # Keep the interest headers
    keep_cols = ["variables", "exp(coef)", "lower .95", "upper .95"]

    for cell in headers_cell:
        if cell.value not in keep_cols:
            col_index = cell.column
            sheet.delete_cols(col_index)


    new_header = ["Variables", "Hazard Ratio", "Lower CI (95%)", "Upper CI (95%)"]

    for i in range(1, len(sheet[1])+1):
        cell_i = sheet.cell(row=1, column=i)
        cell_i.value = new_header[i-1]

    # Update header and adjust column widths
    for i in range(1, len(sheet[1]) + 1):
        cell_i = sheet.cell(row=1, column=i)
        cell_i.value = new_header[i - 1]

        # Get the maximum length of text in the column (excluding header)
        max_length = max(len(str(cell.value)) for cell in sheet[openpyxl.utils.get_column_letter(cell_i.column)][1:])

        # Adjust column width based on the maximum length
        width = max_length
        sheet.column_dimensions[openpyxl.utils.get_column_letter(cell_i.column)].width = width

    # Apply number format to all numbers in columns B to C (excluding first row)
    number_format_style = openpyxl.styles.NamedStyle(name='number_format_style', number_format='0.00')

    # Apply the style to the entire columns C and D
    for column in ['B','C', 'D']:
        for cell in sheet[column][1:]: 
            cell.style = number_format_style

    # Add a header in the top to identify the table
    sheet.insert_rows(1)

    pattern = re.compile(r'Table_\d+_(.*)\.xlsx')
    table_name = pattern.match(excel_name).group(1)
    table_name = re.sub("_", " ", table_name)
    sheet['A1'].value = table_name

    ######### colors #########
    color_green1 = openpyxl.styles.colors.Color(rgb='007932')
    color_green2 = openpyxl.styles.colors.Color(rgb='368f3f')
    color_gray1 = openpyxl.styles.colors.Color(rgb='c9c9c9')
    color_white = openpyxl.styles.colors.Color(rgb='ffffff')

    ######### Set Header formats #########

    # Set name table format
    merge_range = f'A1:{openpyxl.utils.get_column_letter(sheet.max_column)}1'
    sheet.merge_cells(merge_range)

    sheet.row_dimensions[1].height = 40
    sheet["A1"].alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
    sheet["A1"].fill = openpyxl.styles.fills.PatternFill(patternType='solid', fgColor=color_green1)
    sheet["A1"].font = openpyxl.styles.Font(name="SourceSansPro", size=15, bold=True, color=color_white)

    # Set subheaders
    sheet.row_dimensions[2].height = 25
    cell_range = f'A2:{openpyxl.utils.get_column_letter(sheet.max_column)}2'

    for row in sheet[cell_range]:
        for cell in row:
            cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
            cell.fill = openpyxl.styles.fills.PatternFill(patternType='solid', fgColor=color_green2)
            cell.font = openpyxl.styles.Font(name="SourceSansPro", size=11, bold=True, color=color_white)

    ######### Set Body formats #########
    range_cells = f'A3:{openpyxl.utils.get_column_letter(sheet.max_column)}{sheet.max_row}'
    for row in sheet[range_cells]:
        for cell in row:
            if cell.row % 2 == 0:
                cell.fill = openpyxl.styles.fills.PatternFill(patternType='solid', fgColor=color_gray1)
            cell.font = openpyxl.styles.Font(name="SourceSansPro", size=10, bold=False)

    ######### Save #########
    workbook.save(excel_file_path)
    print(f"File {excel_name} correctly formated.")

## 3. Get and transform the files

In [4]:
out_path = "../OUTPUT_figures_tables/"
exel_list = os.listdir(out_path)

pattern = re.compile(r'.*Model.*\.xlsx')
excel_list = [element for element in exel_list if pattern.match(element)]

Iterate over each file

In [5]:
for excel_file in excel_list:
    reformat_excel(out_path, excel_file)

File Table_6_Cox_Model_Report.xlsx correctly formated.
File Table_8_Cox_Model_Report_Over 80 years (Group A).xlsx correctly formated.
File Table_9_Cox_Model_Report_Over 70 years (Group B).xlsx correctly formated.
File Table_11_Cox_Model_Report_Over 40 years (Group D).xlsx correctly formated.
File Table_7_Cox_Model_Refined_Report.xlsx correctly formated.
File Table_13_Cox_Model_Report_Stratification Unprotected, Protected and Incomplete.xlsx correctly formated.
File Table_12_Cox_Model_Report_Over 18 years (Group E).xlsx correctly formated.
File Table_10_Cox_Model_Report_Over 60 years (Group C).xlsx correctly formated.
